# NRC-09: Darks Pipeline Check

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Run the entire pipeline](#dark1_all)
* [Run the individual pipeline steps](#dark1_steps)
   * [Data Quality Initialization](#dq_init)
   * [Saturation Flagging](#saturation)
   * [Superbias Subtraction](#superbias)
   * [Reference Pixel Subtraction](#refpix)
   * [Linearity Correction](#linearity)

<a id='objective'></a>
## Objective

Verify that the pipeline runs the correct calibration steps and produces the correct outputs for all configurations. The [Dark pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_dark.html) applies only some of the basic detector-level corrections from Stage 1 processing to the dark observations. It is applied to one exposure at a time, beginning with an uncalibrated multiaccum ramp (*_uncal.fits file*).

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used:
```python
pip install ipython jupyter matplotlib pylint pandas
```

<a id='imports'></a>
## Imports

In [ ]:
# Packages that allow us to get information about objects:
import asdf
import copy
import os
import shutil
from glob import glob

# Numpy library:
import numpy as np

# Astropy tools:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch

# The pipeline stuff
from jwst.pipeline import calwebb_dark
from jwst.dq_init import DQInitStep
from jwst.saturation import SaturationStep
from jwst.superbias import SuperBiasStep
from jwst.ipc import IPCStep                                                                                    
from jwst.refpix import RefPixStep                                                                
from jwst.linearity import LinearityStep
from jwst import datamodels
from jwst.datamodels import dqflags

import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    """Hilbert's function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel (with a red dot).
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        Image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label='DN')
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

In [ ]:
def side_by_side(data1, data2, vmin, vmax, title1=None, title2=None, title=None):
    """Hilbert's function to show two images side by side for easy comparison. Optionally highlight
    a given pixel with a red dot.
    
    Parameters
    ----------
    data1 : numpy.ndarray
        First image to be displayed
        
    data2 : numpy.ndarray
        Second image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
            
    title1 : str
        Title to use for first (left) plot
        
    title2 : str
        Title to use for the second (right) plot

    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data1, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11, 8))
    im = axes[0].imshow(data1, origin='lower', norm=norm)
    im = axes[1].imshow(data2, origin='lower', norm=norm)
    
    axes[0].set_xlabel('Pixel column')
    axes[0].set_ylabel('Pixel row')
    axes[1].set_xlabel('Pixel column')
    
    if title1:
        axes[0].set_title(title1)
    if title2:
        axes[1].set_title(title2)
        
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax, label='DN')
    
    if title:
        fig.suptitle(title)    

In [ ]:
def plot_ramp(groups, signal, xpixel=None, ypixel=None, title=None):
    """Hilbert's function to plot the up the ramp signal for a pixel.
    
    Parameters
    ----------
    groups : numpy.ndarray
        1D array of group numbers. X-axis values.
        
    signal : numpy.ndarray
        1D array of pixel signal values.
        
    xpixel : int
        X-coordinate of the pixel being plotted. Used for legend only.
        
    ypixel : int
        Y-coordinate of the pixel being plotted. Used for legend only.
        
    title : str
        String to use for the plot title
    """    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    if xpixel and ypixel:
            plt.plot(groups, signal, marker='o',
                     label='Pixel ('+str(xpixel)+','+str(ypixel)+')') 
            plt.legend(loc=2)

    else:
        plt.plot(groups, signal, marker='o')
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

In [ ]:
def plot_ramps(groups, signal1, signal2, label1=None, label2=None, title=None):
    """Hilbert's function to plot the up the ramp signal for two pixels
    on a single plot.
    
    Parameters
    ----------
    groups : numpy.ndarray
        1D array of group numbers. X-axis values.
        
    signal1 : numpy.ndarray
        1D array of signal values for first pixel
        
    signal2 : numpy.ndarray
        1D array of signal values for second pixel
        
    label1 : str
        Label to place in the legend for pixel1
        
    label2 : str
        Label to place in the legend for pixel2
        
    title : str
        String to place in the title of the plot
    """    
    fig = plt.figure(figsize=(6, 6))
    ax = plt.subplot()
    if label1:
        plt.plot(groups, signal1, marker='o', color='black', label=label1)
    else:
        plt.plot(groups, signal1, marker='o', color='black')
    if label2:
        plt.plot(groups, signal2, marker='o', color='red', label=label2)
    else:
        plt.plot(groups, signal2, marker='o', color='red')
    if label1 or label2:
        plt.legend(loc=2)
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    plt.subplots_adjust(top=0.95)
    
    if title:
        plt.title(title)

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
files = glob(base_dir+ground+'/NRC*_484_*uncal.fits')

<a id='dark1_all'></a>
## Run the entire pipeline

In [ ]:
for file in files:
    with datamodels.open(file) as model:
        pipeline = calwebb_dark.DarkPipeline()
        pipeline.refpix.odd_even_rows=False
        pipeline.save_results = True
        pipeline.output_dir = analysis_dir
        pipeline.run(model)

<a id='dark1_steps'></a>
## Run the individual pipeline steps

Choose to run the steps on a file or a list of files. 

In [ ]:
# test_file = files[0]
test_file = '/Users/acanipe/jwebbinars/jw98765001001_01101_00003_nrcb5_uncal.fits'

<a id='dq_init'></a>
### Data Quality Initialization

In [ ]:
with datamodels.open(test_file) as model: 
    dq_init_step = DQInitStep()
    dq_init_step.output_dir = analysis_dir
    dq_init_step.save_results = True

    dq_init = dq_init_step.run(model)

In [ ]:
# Basic information on the number of flagged pixels
idx_pixelDQ = np.where(dq_init.pixeldq.flatten() == 0.)[0]
num_flagged = dq_init.pixeldq.size - len(idx_pixelDQ)
print('Total pixels in PIXELDQ: {}'.format(dq_init.pixeldq.size))
print('{} pixels have no flags.'.format(len(idx_pixelDQ)))
print('{} pixels ({:.2f}% of the detector) have flags.'.format(num_flagged, num_flagged / dq_init.pixeldq.size))

<a id='saturation'></a>
### Saturation Flagging

In [ ]:
saturation_step = SaturationStep()
saturation_step.output_dir = analysis_dir
saturation_step.save_results = True

# Call using the the output from the dq_init step
saturation = saturation_step.run(dq_init)

In [ ]:
# Find indexes of saturated pixels
saturated = np.where(saturation.groupdq & dqflags.pixel['SATURATED'] > 0)
num_sat_flags = len(saturated[0])
print(('Found {} saturated flags. This may include multiple saturated '
       'groups within a given pixel'.format(num_sat_flags)))

<a id='superbias'></a>
### Superbias Subtraction

In [ ]:
superbias_step = SuperBiasStep()
superbias_step.output_dir = analysis_dir
superbias_step.save_results = True

# Call using the the output from the saturation step
superbias = superbias_step.run(saturation)

In [ ]:
# Look at the data before and after superbias subtraction
side_by_side(saturation.data[0, 0, :, :], superbias.data[0, 0, :, :], vmin=10000, vmax=18000,
            title1='Before superbias subtraction', title2='After superbias subtraction')

<a id='refpix'></a>
### Reference Pixel Subtraction

In [ ]:
# Instantiate and set parameters
refpix_step = RefPixStep()
refpix_step.odd_even_rows = False
refpix_step.output_dir = analysis_dir
refpix_step.save_results = True

# Call using the output from the superbias step
refpix = refpix_step.run(superbias)

In [ ]:
# Side by side, before/after reference pixel subtraction
side_by_side(superbias.data[0, 5, 2030:, 1030:1050], refpix.data[0, 5, 2030:, 1030:1050],
             vmin=0, vmax=30000, title1='Before Refpix Subtraction',
             title2='After Refpix Subtraction')

In [ ]:
# Image of the difference before/after reference pixel subtraction
show_image(superbias.data[0, 5, 2030:, 1030:1050] - refpix.data[0, 5, 2030:, 1030:1050],
           vmin=7000, vmax=16000, title='Difference: Before - After Refpix Subtraction')

<a id='linearity'></a>
### Linearity Correction

In [ ]:
# Using the run() method
linearity_step = LinearityStep()
linearity_step.output_dir = analysis_dir
linearity_step.save_results = True

# Call using the refpix output
linearity = linearity_step.run(refpix)

In [ ]:
# Using the 3rd group, find the difference between the data before and after
# linearity correction. Find the pixels where this difference is greater than
# 20 DN, and also where the signal in the final group is over 40,000 DN.
lin_fix = linearity.data[0, 3, :, :] - refpix.data[0, 3, :, :]
well_exposed = np.where((linearity.data[0, -1, :, :] > 40000.) & (lin_fix > 20))

In [ ]:
print('{} pixels meet the criteria above.'.format(len(well_exposed[0])))

In [ ]:
index = 3
lin_pix_x, lin_pix_y = (well_exposed[1][index], well_exposed[0][index])

In [ ]:
# Create an array of group numbers to plot against
group_nums = np.arange(0, refpix.meta.exposure.ngroups)

In [ ]:
plot_ramps(group_nums, refpix.data[0, :, lin_pix_y, lin_pix_x],
           linearity.data[0, :, lin_pix_y, lin_pix_x], label1='Uncorrected', label2='Corrected',
           title='Pixel ({}, {})'.format(lin_pix_x, lin_pix_y))